In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%load_ext line_profiler

In [2]:
import sys
import os
sys.path.append(os.path.abspath('../'))
from api import API
from data import Data
import pandas as pd

pd.set_option('display.max_columns', 150)

## Closest Matching UG Playlist to an ED playlist

In [3]:
d = Data()

In [23]:
diffs, ugPIDs = d.closestMatchingUGPlaylist("Are & Be")

In [25]:
ugPIDs[0]

213590

In [26]:
diffs[0]

1.466596422333839

In [27]:
names, artists = d.getUGPlaylist(213590)

In [28]:
names

['Tiny Little Robots',
 'Shoot to Thrill',
 'Immigrant Song',
 'Welcome to Paradise',
 'Hysteria',
 'Supermassive Black Hole',
 'Uprising',
 "The Kids Aren't Alright",
 'Bad Habit',
 'Coming for You',
 'Tom Sawyer',
 'Brianstorm',
 'N.I.B.',
 'Children of the Grave - Remastered Version',
 "I'm Shipping Up To Boston",
 'The State Of Massachusetts',
 'Paradise City',
 'Welcome To The Jungle',
 'For Whom The Bell Tolls - Remastered',
 'A Moment of Violence',
 'Everything Went Numb',
 'Icky Thump',
 'Ace of Spades',
 'Come Out And Play (Keep ‘Em Separated)',
 'Enter Sandman',
 'The Trooper - 1998 Remastered Version',
 'Hallowed Be Thy Name - 1998 Remastered Version',
 'Holy Wars...The Punishment Due - 2004 Digital Remaster',
 'Black Dragon',
 'Collapse (Post-Amerika)',
 'Woman']

In [29]:
artists

['Cage The Elephant',
 'AC/DC',
 'Led Zeppelin',
 'Green Day',
 'Muse',
 'Muse',
 'Muse',
 'The Offspring',
 'The Offspring',
 'The Offspring',
 'Rush',
 'Arctic Monkeys',
 'Black Sabbath',
 'Black Sabbath',
 'Dropkick Murphys',
 'Dropkick Murphys',
 "Guns N' Roses",
 "Guns N' Roses",
 'Metallica',
 'Streetlight Manifesto',
 'Streetlight Manifesto',
 'The White Stripes',
 'Motörhead',
 'The Offspring',
 'Metallica',
 'Iron Maiden',
 'Iron Maiden',
 'Megadeth',
 'The Vines',
 'Rise Against',
 'Wolfmother']

## Predicting playlist avg features from playlist names

In [8]:
features, names = d.getUGPlaylistFeaturesAndNames()

In [9]:
len(features)

1000000

In [10]:
features[:50]

[[0.7822692307692307,
  0.6592115384615383,
  5.038461538461538,
  -4.8784807692307695,
  0.6923076923076923,
  0.10718846153846154,
  0.08298072884615385,
  0.00067601,
  0.19113846153846153,
  0.6426153846153846,
  123.00753846153846,
  4.0],
 [0.690128205128205,
  0.4947153846153846,
  4.461538461538462,
  -8.288435897435898,
  0.5384615384615384,
  0.08908461538461539,
  0.16404996025641028,
  0.2244309776923077,
  0.18570769230769232,
  0.4754871794871795,
  124.90761538461538,
  3.769230769230769],
 [0.693203125,
  0.671875,
  5.0,
  -4.87415625,
  0.515625,
  0.0962875,
  0.26922984375,
  0.0006378143749999999,
  0.169028125,
  0.565078125,
  114.600671875,
  4.0],
 [0.6212420634920635,
  0.513563492063492,
  5.103174603174603,
  -9.614833333333333,
  0.7142857142857143,
  0.0672015873015873,
  0.27384541428571424,
  0.20205686214285715,
  0.18883412698412697,
  0.45190079365079366,
  125.03249206349206,
  3.9523809523809526],
 [0.6505352941176471,
  0.576764705882353,
  3.35294

In [11]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, Flatten, Activation
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.metrics import mean_squared_error
import re
from sklearn.preprocessing import MinMaxScaler

In [12]:
namesPD = pd.DataFrame(names, columns=['text'])

In [14]:
namesPD.head()

,text
0,Throwbacks
1,Awesome Playlist
2,korean
3,mat
4,90s


In [15]:
scaler = MinMaxScaler()
scaledFeatures = scaler.fit_transform(features)

In [16]:
scaledFeatures[0].shape

(12,)

In [17]:
namesPD.iloc[1000]

text    disney
Name: 1000, dtype: object

In [18]:
namesPD['text'] = namesPD['text'].apply(lambda x: x.lower())
#namesPD['text'] = namesPD['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

In [19]:
max_features = 1750000
tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(namesPD['text'].values)
X = tokenizer.texts_to_sequences(namesPD['text'].values)
X = pad_sequences(X)

In [20]:
X[100000]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
       1319])

In [21]:
scaledFeatures[0]

array([7.82640881e-01, 6.97397663e-01, 4.83474137e-01, 9.15080042e-01,
       6.92307692e-01, 1.00018388e-01, 8.33883723e-02, 7.05647182e-04,
       1.74221768e-01, 6.56053906e-01, 6.11817686e-01, 8.63636364e-01])

In [22]:
embed_dim = 128
lstm_out = 150

model = Sequential()
model.add(Embedding(max_features, embed_dim, input_length = X.shape[1]))
#model.add(SpatialDropout1D(0.4))
model.add(Flatten())
#model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(lstm_out))
model.add(Activation('relu'))
model.add(Dense(lstm_out))
model.add(Activation('relu'))
model.add(Dense(12))
model.compile(loss = 'mean_squared_error', optimizer='adam', metrics=['mse'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 12, 128)           224000000 
_________________________________________________________________
flatten_1 (Flatten)          (None, 1536)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 150)               230550    
_________________________________________________________________
activation_1 (Activation)    (None, 150)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 150)               22650     
_________________________________________________________________
activation_2 (Activation)    (None, 150)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 12)                1812      
Total para

In [23]:
scaledFeatures.shape

(1000000, 12)

In [24]:
X.shape

(1000000, 12)

In [25]:
Y = scaledFeatures
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(670000, 12) (670000, 12)
(330000, 12) (330000, 12)


In [26]:
batch_size = 4092
model.fit(X_train, Y_train, epochs = 4, batch_size=batch_size, verbose = 2)

C:\Users\Adam Khazi\AppData\Local\conda\conda\envs\py36\lib\site-packages\tensorflow\python\ops\gradients_impl.py:97: UserWarning: Converting sparse IndexedSlices to a dense Tensor with 224000000 elements. This may consume a large amount of memory.
  num_elements)


Epoch 1/4
 - 648s - loss: 0.0179 - mean_squared_error: 0.0179
Epoch 2/4
 - 617s - loss: 0.0072 - mean_squared_error: 0.0072
Epoch 3/4
 - 623s - loss: 0.0069 - mean_squared_error: 0.0069
Epoch 4/4
 - 623s - loss: 0.0068 - mean_squared_error: 0.0068


In [27]:
validation_size = 15000

X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.5f" % (score))
print("acc: %.5f" % (acc))

score: 0.00695
acc: 0.00695


In [32]:
X_test[3]

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 22])

In [28]:
Y_preds = model.predict(X_test)

In [33]:
Y_test

array([[0.72374213, 0.57470808, 0.60019268, ..., 0.43706233, 0.61944413,
        0.85653409],
       [0.13596843, 0.24215165, 0.46914315, ..., 0.09094576, 0.48358304,
        0.78676471],
       [0.60505542, 0.77872767, 0.50509221, ..., 0.43259559, 0.63884413,
        0.85974026],
       ...,
       [0.65427118, 0.46511391, 0.43848059, ..., 0.25905563, 0.66285239,
        0.83116883],
       [0.49796627, 0.69586045, 0.56693275, ..., 0.46847224, 0.50328583,
        0.85281385],
       [0.73375751, 0.58773347, 0.49459019, ..., 0.53633612, 0.6524755 ,
        0.8548951 ]])

In [34]:
Y_preds

array([[0.76064134, 0.57123375, 0.49219283, ..., 0.4614572 , 0.6385947 ,
        0.8494069 ],
       [0.44554296, 0.51522493, 0.487983  , ..., 0.3076586 , 0.5643487 ,
        0.83283615],
       [0.67966217, 0.6432028 , 0.49449462, ..., 0.5130906 , 0.60469127,
        0.8525846 ],
       ...,
       [0.5664659 , 0.48961118, 0.49574414, ..., 0.3412629 , 0.59895   ,
        0.831091  ],
       [0.5125895 , 0.63165826, 0.49677017, ..., 0.42639726, 0.5673312 ,
        0.8409943 ],
       [0.69889337, 0.60843694, 0.5149358 , ..., 0.57931185, 0.63331825,
        0.8501437 ]], dtype=float32)

In [89]:
Y_test.mean()

0.4829267819650614

In [29]:
Y_preds.std()

0.27548656

In [99]:
from sklearn.metrics import mean_squared_error
mean_squared_error(Y_test, Y_preds)

0.006894669521572548

In [35]:
0.006894669521572548 / (X_test.max() - X_test.min())

6.286168418647473e-07

## Playlist Track Classifier

In [4]:
import numpy as np

In [81]:
posFeatures, negFeatures = d.getPlaylistWTrackVectorsRefined(500, 10)

100%|████████████████████████████████████████████████████████████████████████████████| 500/500 [00:10<00:00, 47.92it/s]


In [ ]:
#%lprun -f d.getPlaylistWTrackVectorsRefined d.getPlaylistWTrackVectorsRefined(25000, 5)

In [82]:
posFeatures[0]

{'danceability': 0.904,
 'energy': 0.813,
 'key': 4,
 'loudness': -7.105,
 'mode': 0,
 'speechiness': 0.121,
 'acousticness': 0.0311,
 'instrumentalness': 0.00697,
 'liveness': 0.0471,
 'valence': 0.81,
 'tempo': 125.461,
 'duration_ms': 226863,
 'time_signature': 4,
 'popularity': 65,
 'release_date': '2005',
 'album_name': 'The Cookbook',
 'artist_name': 'Missy Elliott',
 'pos': 0,
 'track_name': 'Lose Control (feat. Ciara & Fat Man Scoop)',
 'playlist_collaborative': 'false',
 'playlist_description': 'nan',
 'playlist_duration_ms': 11532414,
 'playlist_modified_at': 1493424000,
 'playlist_name': 'Throwbacks',
 'playlist_num_albums': 47,
 'playlist_num_artists': 37,
 'playlist_num_edits': 6,
 'playlist_num_followers': 1,
 'playlist_num_tracks': 52}

In [83]:
#posFeatures, negFeatures = d.getPlaylistWTrackVectors(200000)

In [84]:
allFeatures = posFeatures + negFeatures

In [85]:
len(allFeatures)

9938

In [86]:
Y = [1 for i in range(len(posFeatures))] + [0 for i in range(len(negFeatures))]

In [87]:
from random import shuffle

In [88]:
shuffle(allFeatures)

In [89]:
c = list(zip(allFeatures, Y))
shuffle(c)
allFeatures, Y = zip(*c)
allFeatures = list(allFeatures)
Y = list(Y)

In [90]:
len(Y)

9938

In [91]:
negFeatures[0]

{'pos': 63,
 'playlist_collaborative': 'false',
 'playlist_description': 'nan',
 'playlist_duration_ms': 11532414,
 'playlist_modified_at': 1493424000,
 'playlist_name': 'Throwbacks',
 'playlist_num_albums': 47,
 'playlist_num_artists': 37,
 'playlist_num_edits': 6,
 'playlist_num_followers': 1,
 'playlist_num_tracks': 52,
 'danceability': 0.596,
 'energy': 0.463,
 'key': 1,
 'loudness': -15.969,
 'mode': 1,
 'speechiness': 0.0619,
 'acousticness': 0.316,
 'instrumentalness': 0.76,
 'liveness': 0.121,
 'valence': 0.441,
 'tempo': 131.503,
 'duration_ms': 280023,
 'time_signature': 4,
 'popularity': 7,
 'release_date': '2006',
 'album_name': 'Song Islands Vol. 2',
 'artist_name': 'Mount Eerie',
 'track_name': 'The Intimacy'}

In [92]:
featuresDF = pd.DataFrame(allFeatures)

In [93]:
featuresDF.columns

Index(['acousticness', 'album_name', 'artist_name', 'danceability',
       'duration_ms', 'energy', 'instrumentalness', 'key', 'liveness',
       'loudness', 'mode', 'playlist_collaborative', 'playlist_description',
       'playlist_duration_ms', 'playlist_modified_at', 'playlist_name',
       'playlist_num_albums', 'playlist_num_artists', 'playlist_num_edits',
       'playlist_num_followers', 'playlist_num_tracks', 'popularity', 'pos',
       'release_date', 'speechiness', 'tempo', 'time_signature', 'track_name',
       'valence'],
      dtype='object')

In [94]:
textCols = ['album_name', 'artist_name', 'playlist_description', 'playlist_name', 'track_name']
numericCols = ['acousticness', 'danceability',
       'duration_ms', 'energy', 'instrumentalness', 'liveness',
       'loudness',
       'playlist_duration_ms', 'playlist_modified_at',
       'playlist_num_albums', 'playlist_num_artists', 'playlist_num_edits',
       'playlist_num_followers', 'playlist_num_tracks', 'popularity',
       'speechiness', 'tempo', 'valence']

In [95]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [96]:
featuresDF[numericCols].head(5)

,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,playlist_duration_ms,playlist_modified_at,playlist_num_albums,playlist_num_artists,playlist_num_edits,playlist_num_followers,playlist_num_tracks,popularity,speechiness,tempo,valence
0,0.2530,0.477,213151,0.655,0.0000,0.0741,-9.660,5684090,1508457600,26,23,25,1,27,13.0,0.0385,137.387,0.671
1,0.2030,0.316,318040,0.367,0.0665,0.2430,-9.253,24126961,1473379200,88,72,8,1,111,11.0,0.0261,103.111,0.125
2,0.0453,0.603,268306,0.827,0.6190,0.0833,-7.372,8787620,1472947200,39,34,8,2,41,60.0,0.0714,144.992,0.311
3,0.2200,0.625,248156,0.707,0.0000,0.2130,-4.761,15247117,1507593600,58,52,56,1,70,64.0,0.0337,138.064,0.479
4,0.0147,0.488,216546,0.967,0.0000,0.1090,-2.701,19672852,1506816000,57,36,14,1,82,47.0,0.0755,130.926,0.341


In [97]:
delNans = list(featuresDF.index[featuresDF[numericCols].isnull().T.any().T])
delNans

[2218, 5464, 6224]

In [98]:
#featuresDF[numericCols].iloc[876]

In [99]:
for i in range(len(delNans)-1, -1, -1):
    Y.pop(delNans[i])

In [100]:
featuresDF.iloc[0]

acousticness                                             0.253
album_name                The Best of Johnny Clegg 1979 - 2016
artist_name                                       Johnny Clegg
danceability                                             0.477
duration_ms                                             213151
energy                                                   0.655
instrumentalness                                             0
key                                                          2
liveness                                                0.0741
loudness                                                 -9.66
mode                                                         0
playlist_collaborative                                   false
playlist_description                                       nan
playlist_duration_ms                                   5684090
playlist_modified_at                                1508457600
playlist_name                                          

In [101]:
delNans

[2218, 5464, 6224]

In [102]:
# drop rows with nan in them
featuresDF = featuresDF.drop(featuresDF.index[featuresDF[numericCols].isnull().T.any().T])

In [103]:
print(len(Y), len(featuresDF))

9935 9935


In [104]:
# checking types of all columns
featuresDF.dtypes

acousticness              float64
album_name                 object
artist_name                object
danceability              float64
duration_ms                 int64
energy                    float64
instrumentalness          float64
key                         int64
liveness                  float64
loudness                  float64
mode                        int64
playlist_collaborative     object
playlist_description       object
playlist_duration_ms        int64
playlist_modified_at        int64
playlist_name              object
playlist_num_albums         int64
playlist_num_artists        int64
playlist_num_edits          int64
playlist_num_followers      int64
playlist_num_tracks         int64
popularity                float64
pos                         int64
release_date               object
speechiness               float64
tempo                     float64
time_signature              int64
track_name                 object
valence                   float64
dtype: object

In [105]:
featuresDF[numericCols] = scaler.fit_transform(featuresDF[numericCols])

In [106]:
# checking types of all columns -- release date is the wrong type
featuresDF.dtypes

acousticness              float64
album_name                 object
artist_name                object
danceability              float64
duration_ms               float64
energy                    float64
instrumentalness          float64
key                         int64
liveness                  float64
loudness                  float64
mode                        int64
playlist_collaborative     object
playlist_description       object
playlist_duration_ms      float64
playlist_modified_at      float64
playlist_name              object
playlist_num_albums       float64
playlist_num_artists      float64
playlist_num_edits        float64
playlist_num_followers    float64
playlist_num_tracks       float64
popularity                float64
pos                         int64
release_date               object
speechiness               float64
tempo                     float64
time_signature              int64
track_name                 object
valence                   float64
dtype: object

In [107]:
featuresDF['release_date']  = pd.to_numeric(featuresDF['release_date'])

In [108]:
import nltk

In [109]:
def preprocess(t):
    t = t.lower()
    return nltk.word_tokenize(t)

In [110]:
for t in textCols:
    featuresDF[t] = featuresDF.apply(lambda row: preprocess(row[t]), axis=1)

In [111]:
featuresDF.head()

,acousticness,album_name,artist_name,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,playlist_collaborative,playlist_description,playlist_duration_ms,playlist_modified_at,playlist_name,playlist_num_albums,playlist_num_artists,playlist_num_edits,playlist_num_followers,playlist_num_tracks,popularity,pos,release_date,speechiness,tempo,time_signature,track_name,valence
0,0.254016,"[the, best, of, johnny, clegg, 1979, -, 2016]","[johnny, clegg]",0.486735,0.045494,0.655,0.000000,2,0.074773,0.802871,0,false,[nan],0.082394,0.994452,[indie],0.107623,0.119048,0.181102,0.000000,0.091667,0.141304,44,2016,0.040021,0.641645,4,"[third, world, child, -, '13]",0.672345
1,0.203815,"[(, a, ), spera]",[mirah],0.322449,0.067903,0.367,0.066767,7,0.245207,0.809362,1,false,[nan],0.407011,0.806750,"[car, rides]",0.385650,0.410714,0.047244,0.000000,0.441667,0.119565,14,2009,0.027131,0.481564,3,[education],0.125251
2,0.045482,"[parallel, paradise]",[hermitude],0.615306,0.057278,0.827,0.621486,6,0.084057,0.839362,1,false,[nan],0.137020,0.804438,"[on, the, run]",0.165919,0.184524,0.047244,0.047619,0.150000,0.652174,14,2012,0.074220,0.677162,4,"[hyperparadise, -, flume, remix]",0.311623
3,0.220884,"[feel, good, (, feat, ., daya, )]",[gryffin],0.637755,0.052973,0.707,0.000000,2,0.214934,0.881005,1,false,[nan],0.250715,0.989829,"[good, vibes]",0.251121,0.291667,0.425197,0.000000,0.270833,0.695652,65,2017,0.035031,0.644806,4,"[feel, good, (, feat, ., daya, )]",0.479960
4,0.014759,"[so, wrong, ,, it, 's, right]","[all, time, low]",0.497959,0.046219,0.967,0.000000,3,0.109990,0.913860,1,false,[nan],0.328613,0.985668,[chill],0.246637,0.196429,0.094488,0.000000,0.320833,0.510870,16,2007,0.078482,0.611469,4,"[six, feet, under, the, stars]",0.341683


In [36]:
import pandas as pd
import csv

embedding_path = "F:\ML Data\glove.840B.300d\glove.840B.300d.txt"
import pandas as pd
import csv

words = pd.read_table(embedding_path, sep=" ", index_col=0, header=None, quoting=csv.QUOTE_NONE)

In [112]:
def vec(w):
  return words.loc[w].as_matrix()

In [113]:
from keras.preprocessing.sequence import pad_sequences
from tqdm import tqdm

import pdb
#pad_sequences([[1, 2, 3], [3, 4, 5, 6], [7, 8]], padding='post')

def convertV(w):
    V = []
    for i in w:
        if i in words.index:
            V.append(vec(i))
    return np.concatenate(V) if V else np.zeros(1)

# replace with word embeddings for each word concatenated    
for t in textCols:
    featuresDF[t] = featuresDF.apply(lambda row: convertV(row[t]), axis=1)

C:\Users\Adam Khazi\AppData\Local\conda\conda\envs\py36\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


In [114]:
frames = []
for t in textCols:
    temp = pd.DataFrame(data=pad_sequences(featuresDF[t].values, padding='post', dtype='float64')).add_prefix(t+'_')
    print(temp.shape)
    frames.append(temp)

print("featuresDF", featuresDF.shape)
featuresDF = featuresDF.drop(textCols, axis=1)
newFeaturesDF = pd.concat([featuresDF] + frames, axis=1)
print("newFeaturesDF", newFeaturesDF.shape)

(9935, 14400)
(9935, 5700)
(9935, 2400)
(9935, 2700)
(9935, 11400)
featuresDF (9935, 29)
newFeaturesDF (9938, 36624)


In [115]:
newFeaturesDF.head(100)

,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,playlist_collaborative,playlist_duration_ms,playlist_modified_at,playlist_num_albums,playlist_num_artists,playlist_num_edits,playlist_num_followers,playlist_num_tracks,popularity,pos,release_date,speechiness,tempo,time_signature,valence,album_name_0,album_name_1,album_name_2,album_name_3,album_name_4,album_name_5,album_name_6,album_name_7,album_name_8,album_name_9,album_name_10,album_name_11,album_name_12,album_name_13,album_name_14,album_name_15,album_name_16,album_name_17,album_name_18,album_name_19,album_name_20,album_name_21,album_name_22,album_name_23,album_name_24,album_name_25,album_name_26,album_name_27,album_name_28,album_name_29,album_name_30,album_name_31,album_name_32,album_name_33,album_name_34,album_name_35,album_name_36,album_name_37,album_name_38,album_name_39,album_name_40,album_name_41,album_name_42,album_name_43,album_name_44,album_name_45,album_name_46,album_name_47,album_name_48,album_name_49,album_name_50,...,track_name_11325,track_name_11326,track_name_11327,track_name_11328,track_name_11329,track_name_11330,track_name_11331,track_name_11332,track_name_11333,track_name_11334,track_name_11335,track_name_11336,track_name_11337,track_name_11338,track_name_11339,track_name_11340,track_name_11341,track_name_11342,track_name_11343,track_name_11344,track_name_11345,track_name_11346,track_name_11347,track_name_11348,track_name_11349,track_name_11350,track_name_11351,track_name_11352,track_name_11353,track_name_11354,track_name_11355,track_name_11356,track_name_11357,track_name_11358,track_name_11359,track_name_11360,track_name_11361,track_name_11362,track_name_11363,track_name_11364,track_name_11365,track_name_11366,track_name_11367,track_name_11368,track_name_11369,track_name_11370,track_name_11371,track_name_11372,track_name_11373,track_name_11374,track_name_11375,track_name_11376,track_name_11377,track_name_11378,track_name_11379,track_name_11380,track_name_11381,track_name_11382,track_name_11383,track_name_11384,track_name_11385,track_name_11386,track_name_11387,track_name_11388,track_name_11389,track_name_11390,track_name_11391,track_name_11392,track_name_11393,track_name_11394,track_name_11395,track_name_11396,track_name_11397,track_name_11398,track_name_11399
0,0.254016,0.486735,0.045494,0.6550,0.000000,2.0,0.074773,0.802871,0.0,false,0.082394,0.994452,0.107623,0.119048,0.181102,0.000000,0.091667,0.141304,44.0,2016.0,0.040021,0.641645,4.0,0.672345,0.272040,-0.062030,-0.188400,0.023225,-0.018158,0.006719,-0.138770,0.177080,0.177090,2.588200,-0.351790,-0.173120,0.432850,-0.107080,0.150060,-0.199820,-0.190930,1.18710,-0.162070,-0.235380,0.003664,-0.191560,-0.085662,0.039199,-0.066449,-0.042090,-0.191220,0.011679,-0.371380,0.218860,0.001142,0.431900,-0.142050,0.380590,0.306540,0.020167,-0.183160,-0.006519,-0.008055,-0.120630,0.027507,0.298390,-0.228960,-0.228820,0.146710,-0.076301,-0.126800,-0.006665,-0.052795,0.142580,0.156100,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.203815,0.322449,0.067903,0.3670,0.066767,7.0,0.245207,0.809362,1.0,false,0.407011,0.806750,0.385650,0.410714,0.047244,0.000000,0.441667,0.119565,14.0,2009.0,0.027131,0.481564,3.0,0.125251,-0.180240,0.008411,-0.304630,0.209970,0.085153,0.033191,-0.149680,-0.324530,0.206750,0.946890,0.076250,0.504420,0.167390,-0.415820,0.098140,0.379840,-0.137180,1.33650,-0.603220,-0.135120,-0.256020,-0.287170,-0.179370,0.319300,0.190850,0.029798,-0.116520,-0.313510,0.055655,0.033468,0.058227,0.419480,-0.330610,0.154110,0.330220,0.288720,0.184500,0.490950,0.155660,0.190960,0.072315,-0.054701,-0.018773,-0.098779,0.005156,-0.263030,-0.028072,0.323560,0.392820,0.225470,0.215960,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

In [129]:
newFeaturesDF.iloc[9937]

acousticness                       0.568273
danceability                       0.537755
duration_ms                        0.036090
energy                             0.587000
instrumentalness                   0.000000
key                                5.000000
liveness                           0.218971
loudness                           0.899601
mode                               1.000000
playlist_duration_ms               0.190804
playlist_modified_at               0.497920
playlist_num_albums                0.197309
playlist_num_artists               0.154762
playlist_num_edits                 0.165354
playlist_num_followers             0.047619
playlist_num_tracks                0.229167
popularity                         0.130435
pos                                5.000000
release_date                    2008.000000
speechiness                        0.090852
tempo                              0.963058
time_signature                     4.000000
valence                         

In [116]:
newFeaturesDF.columns

Index(['acousticness', 'danceability', 'duration_ms', 'energy',
       'instrumentalness', 'key', 'liveness', 'loudness', 'mode',
       'playlist_collaborative',
       ...
       'track_name_11390', 'track_name_11391', 'track_name_11392',
       'track_name_11393', 'track_name_11394', 'track_name_11395',
       'track_name_11396', 'track_name_11397', 'track_name_11398',
       'track_name_11399'],
      dtype='object', length=36624)

In [117]:
newFeaturesDF["playlist_description_0"]

0       0.00000
1       0.00000
2       0.00000
3       0.00000
4       0.00000
5       0.00000
6       0.00000
7       0.00000
8       0.00000
9       0.00000
10      0.00000
11      0.00000
12      0.00000
13      0.00000
14      0.00000
15      0.00000
16      0.00000
17      0.00000
18      0.00000
19      0.00000
20      0.00000
21      0.00000
22      0.00000
23      0.00000
24      0.00000
25      0.00000
26      0.00000
27      0.00000
28      0.00000
29      0.00000
         ...   
9908    0.00000
9909    0.00000
9910    0.00000
9911    0.00000
9912    0.00000
9913    0.00000
9914    0.00000
9915    0.00000
9916    0.00000
9917    0.00000
9918    0.00000
9919   -0.03359
9920    0.00000
9921    0.00000
9922    0.00000
9923    0.00000
9924    0.00000
9925    0.00000
9926    0.00000
9927    0.00000
9928    0.00000
9929    0.00000
9930    0.00000
9931    0.00000
9932    0.00000
9933    0.00000
9934    0.00000
9935        NaN
9936        NaN
9937        NaN
Name: playlist_descripti

In [118]:
newFeaturesDF.columns

Index(['acousticness', 'danceability', 'duration_ms', 'energy',
       'instrumentalness', 'key', 'liveness', 'loudness', 'mode',
       'playlist_collaborative',
       ...
       'track_name_11390', 'track_name_11391', 'track_name_11392',
       'track_name_11393', 'track_name_11394', 'track_name_11395',
       'track_name_11396', 'track_name_11397', 'track_name_11398',
       'track_name_11399'],
      dtype='object', length=36624)

In [119]:
newFeaturesDF = pd.get_dummies(newFeaturesDF, columns=["playlist_collaborative"])

In [147]:
newFeaturesDF = newFeaturesDF.fillna(value=0)

In [120]:
newFeaturesDF.head()

,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,playlist_duration_ms,playlist_modified_at,playlist_num_albums,playlist_num_artists,playlist_num_edits,playlist_num_followers,playlist_num_tracks,popularity,pos,release_date,speechiness,tempo,time_signature,valence,album_name_0,album_name_1,album_name_2,album_name_3,album_name_4,album_name_5,album_name_6,album_name_7,album_name_8,album_name_9,album_name_10,album_name_11,album_name_12,album_name_13,album_name_14,album_name_15,album_name_16,album_name_17,album_name_18,album_name_19,album_name_20,album_name_21,album_name_22,album_name_23,album_name_24,album_name_25,album_name_26,album_name_27,album_name_28,album_name_29,album_name_30,album_name_31,album_name_32,album_name_33,album_name_34,album_name_35,album_name_36,album_name_37,album_name_38,album_name_39,album_name_40,album_name_41,album_name_42,album_name_43,album_name_44,album_name_45,album_name_46,album_name_47,album_name_48,album_name_49,album_name_50,album_name_51,...,track_name_11327,track_name_11328,track_name_11329,track_name_11330,track_name_11331,track_name_11332,track_name_11333,track_name_11334,track_name_11335,track_name_11336,track_name_11337,track_name_11338,track_name_11339,track_name_11340,track_name_11341,track_name_11342,track_name_11343,track_name_11344,track_name_11345,track_name_11346,track_name_11347,track_name_11348,track_name_11349,track_name_11350,track_name_11351,track_name_11352,track_name_11353,track_name_11354,track_name_11355,track_name_11356,track_name_11357,track_name_11358,track_name_11359,track_name_11360,track_name_11361,track_name_11362,track_name_11363,track_name_11364,track_name_11365,track_name_11366,track_name_11367,track_name_11368,track_name_11369,track_name_11370,track_name_11371,track_name_11372,track_name_11373,track_name_11374,track_name_11375,track_name_11376,track_name_11377,track_name_11378,track_name_11379,track_name_11380,track_name_11381,track_name_11382,track_name_11383,track_name_11384,track_name_11385,track_name_11386,track_name_11387,track_name_11388,track_name_11389,track_name_11390,track_name_11391,track_name_11392,track_name_11393,track_name_11394,track_name_11395,track_name_11396,track_name_11397,track_name_11398,track_name_11399,playlist_collaborative_false,playlist_collaborative_true
0,0.254016,0.486735,0.045494,0.655,0.000000,2.0,0.074773,0.802871,0.0,0.082394,0.994452,0.107623,0.119048,0.181102,0.000000,0.091667,0.141304,44.0,2016.0,0.040021,0.641645,4.0,0.672345,0.272040,-0.062030,-0.18840,0.023225,-0.018158,0.006719,-0.138770,0.17708,0.177090,2.58820,-0.351790,-0.173120,0.432850,-0.10708,0.15006,-0.19982,-0.190930,1.1871,-0.16207,-0.235380,0.003664,-0.191560,-0.085662,0.039199,-0.066449,-0.042090,-0.19122,0.011679,-0.371380,0.218860,0.001142,0.43190,-0.142050,0.380590,0.306540,0.020167,-0.183160,-0.006519,-0.008055,-0.120630,0.027507,0.298390,-0.228960,-0.228820,0.146710,-0.076301,-0.126800,-0.006665,-0.052795,0.142580,0.156100,0.055510,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
1,0.203815,0.322449,0.067903,0.367,0.066767,7.0,0.245207,0.809362,1.0,0.407011,0.806750,0.385650,0.410714,0.047244,0.000000,0.441667,0.119565,14.0,2009.0,0.027131,0.481564,3.0,0.125251,-0.180240,0.008411,-0.30463,0.209970,0.085153,0.033191,-0.149680,-0.32453,0.206750,0.94689,0.076250,0.504420,0.167390,-0.41582,0.09814,0.37984,-0.137180,1.3365,-0.60322,-0.135120,-0.256020,-0.287170,-0.179370,0.319300,0.190850,0.029798,-0.11652,-0.313510,0.055655,0.033468,0.058227,0.41948,-0.330610,0.154110,0.330220,0.288720,0.184500,0.490950,0.155660,0.190960,0.072315,-0.054701,-0.018773,-0.098779,0.005156,-0.263030,-0.028072,0.323560,0.392820,0.225470,0.215960,-0.083058,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

In [121]:
newFeatures = newFeaturesDF.values

In [122]:
newFeatures.shape

(9938, 36625)

In [130]:
newFeatures = newFeatures[:9930]

In [131]:
Y = Y[:9930]

In [136]:
newFeaturesDF = newFeaturesDF.loc[:9929]

In [137]:
newFeaturesDF.shape

(9930, 36625)

In [148]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(newFeaturesDF, Y, test_size = 0.20)

In [149]:
type(X_train.values)

numpy.ndarray

In [140]:
X_train

,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,playlist_duration_ms,playlist_modified_at,playlist_num_albums,playlist_num_artists,playlist_num_edits,playlist_num_followers,playlist_num_tracks,popularity,pos,release_date,speechiness,tempo,time_signature,valence,album_name_0,album_name_1,album_name_2,album_name_3,album_name_4,album_name_5,album_name_6,album_name_7,album_name_8,album_name_9,album_name_10,album_name_11,album_name_12,album_name_13,album_name_14,album_name_15,album_name_16,album_name_17,album_name_18,album_name_19,album_name_20,album_name_21,album_name_22,album_name_23,album_name_24,album_name_25,album_name_26,album_name_27,album_name_28,album_name_29,album_name_30,album_name_31,album_name_32,album_name_33,album_name_34,album_name_35,album_name_36,album_name_37,album_name_38,album_name_39,album_name_40,album_name_41,album_name_42,album_name_43,album_name_44,album_name_45,album_name_46,album_name_47,album_name_48,album_name_49,album_name_50,album_name_51,...,track_name_11327,track_name_11328,track_name_11329,track_name_11330,track_name_11331,track_name_11332,track_name_11333,track_name_11334,track_name_11335,track_name_11336,track_name_11337,track_name_11338,track_name_11339,track_name_11340,track_name_11341,track_name_11342,track_name_11343,track_name_11344,track_name_11345,track_name_11346,track_name_11347,track_name_11348,track_name_11349,track_name_11350,track_name_11351,track_name_11352,track_name_11353,track_name_11354,track_name_11355,track_name_11356,track_name_11357,track_name_11358,track_name_11359,track_name_11360,track_name_11361,track_name_11362,track_name_11363,track_name_11364,track_name_11365,track_name_11366,track_name_11367,track_name_11368,track_name_11369,track_name_11370,track_name_11371,track_name_11372,track_name_11373,track_name_11374,track_name_11375,track_name_11376,track_name_11377,track_name_11378,track_name_11379,track_name_11380,track_name_11381,track_name_11382,track_name_11383,track_name_11384,track_name_11385,track_name_11386,track_name_11387,track_name_11388,track_name_11389,track_name_11390,track_name_11391,track_name_11392,track_name_11393,track_name_11394,track_name_11395,track_name_11396,track_name_11397,track_name_11398,track_name_11399,playlist_collaborative_false,playlist_collaborative_true
1524,0.441767,0.672449,0.054492,0.6340,0.000000,1.0,0.179617,0.838070,0.0,0.167025,0.790569,0.130045,0.119048,0.220472,0.000000,0.154167,0.380435,21.0,2015.0,0.407484,0.729919,4.0,0.500000,-0.595290,0.828300,-0.071671,-0.151270,0.194200,0.154960,0.190430,0.451500,0.000839,1.83860,0.080270,-0.398890,0.066208,0.149000,-0.177090,0.182050,0.176540,2.230100,-0.439780,0.168230,-0.050348,-0.324740,-0.197780,-0.382320,-0.319940,0.299550,-0.116270,0.084962,-0.302500,-0.005906,0.347420,0.257140,0.117980,-0.308850,0.207400,-0.500570,-0.451480,-0.014536,0.157440,0.143400,0.364260,-0.102900,0.196050,0.010951,0.359000,-0.071260,-0.446480,-0.075739,-0.172280,-0.114900,-0.289150,0.310920,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
3300,0.743976,0.742857,0.043014,0.7300,0.000000,5.0,0.711403,0.870781,1.0,0.101670,0.995839,0.062780,0.053571,0.047244,0.000000,0.100000,0.663043,18.0,2017.0,0.221414,0.635410,4.0,0.737475,-0.439450,-0.271970,-0.012538,-0.799050,0.056359,0.457540,0.417610,-0.085828,-0.156200,1.12040,-0.010525,0.307940,0.213650,0.061002,-0.388940,-0.143420,-0.007605,0.553710,-0.229700,0.177850,0.062105,-0.022384,-0.329660,0.358710,0.425540,-0.442000,-0.344290,-0.324040,0.921680,-0.690800,-0.483470,0.169320,0.248880,-0.057993,-0.051126,-0.175380,0.419960,0.201210,-0.598820,-0.083457,-0.556400,0.418240,-0.310710,0.137360,-0.079710,0.413250,0.640130,-0.290480,-0.319630,0.172850,0.233840,0.521960,...,0.0,0.0

In [150]:
import scipy
X_train_sparse = scipy.sparse.csr_matrix(X_train.values)

In [151]:
X_test_sparse = scipy.sparse.csr_matrix(X_test.values)

In [152]:
from keras.models import Sequential
from keras.layers import Dense, Dropout

In [185]:
# create model
model = Sequential()
model.add(Dense(400, input_dim=X_train.shape[1], kernel_initializer='he_normal', activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(250, kernel_initializer='he_normal', activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(100, kernel_initializer='he_normal', activation='relu'))
model.add(Dense(1, kernel_initializer='he_normal', activation='sigmoid'))
# Compile model. We use the the logarithmic loss function, and the Adam gradient optimizer.
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [186]:
batch_size = 128
model.fit(X_train, Y_train, epochs = 100, batch_size=batch_size, verbose = 2)

Epoch 1/100
 - 29s - loss: 2.9524 - acc: 0.4987
Epoch 2/100
 - 27s - loss: 0.7758 - acc: 0.4922
Epoch 3/100
 - 27s - loss: 0.7023 - acc: 0.4986
Epoch 4/100
 - 27s - loss: 0.6942 - acc: 0.5026
Epoch 5/100
 - 27s - loss: 0.6936 - acc: 0.4960
Epoch 6/100
 - 27s - loss: 0.6948 - acc: 0.4930
Epoch 7/100
 - 27s - loss: 0.6928 - acc: 0.4992
Epoch 8/100
 - 27s - loss: 0.6923 - acc: 0.5083
Epoch 9/100
 - 27s - loss: 0.6919 - acc: 0.5112
Epoch 10/100
 - 27s - loss: 0.6924 - acc: 0.5055
Epoch 11/100
 - 27s - loss: 0.6922 - acc: 0.5031
Epoch 12/100
 - 27s - loss: 0.6917 - acc: 0.5116
Epoch 13/100
 - 27s - loss: 0.6913 - acc: 0.5058
Epoch 14/100
 - 27s - loss: 0.6909 - acc: 0.5097
Epoch 15/100
 - 27s - loss: 0.6912 - acc: 0.5082
Epoch 16/100
 - 27s - loss: 0.6899 - acc: 0.5123
Epoch 17/100
 - 27s - loss: 0.6895 - acc: 0.5040
Epoch 18/100
 - 27s - loss: 0.6883 - acc: 0.5101
Epoch 19/100
 - 27s - loss: 0.6909 - acc: 0.5078
Epoch 20/100
 - 27s - loss: 0.6901 - acc: 0.5091
Epoch 21/100
 - 27s - loss: 0

In [187]:
# predict on training set to make sure code works
x_train_pred = model.predict(X_train)
x_train_pred = (x_train_pred > 0.5)
accuracy_score(Y_train, x_train_pred)

0.6174471299093656

In [188]:
y_pred = model.predict(X_test)

In [189]:
y_pred

array([[0.5532047],
       [0.5532047],
       [0.5532047],
       ...,
       [0.5532047],
       [0.5532047],
       [0.5532047]], dtype=float32)

In [190]:
y_pred = (y_pred > 0.5)

In [191]:
from sklearn.metrics import accuracy_score
accuracy_score(Y_test, y_pred)

0.5105740181268882

In [ ]:
## hierarchical model
sentence_input = Input(shape=(MAX_SENT_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sentence_input)
l_lstm = Bidirectional(LSTM(100))(embedded_sequences)
sentEncoder = Model(sentence_input, l_lstm)

review_input = Input(shape=(MAX_SENTS,MAX_SENT_LENGTH), dtype='int32')
review_encoder = TimeDistributed(sentEncoder)(review_input)
l_lstm_sent = Bidirectional(LSTM(100))(review_encoder)
preds = Dense(2, activation='softmax')(l_lstm_sent)
model = Model(review_input, preds)

In [151]:
## SVMS

In [162]:
from sklearn import svm

clf = svm.SVC()
clf.fit(X_train_sparse, Y_train)  

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [163]:
Y_test_preds = clf.predict(X_test_sparse)

In [157]:
from sklearn import linear_model
logreg = linear_model.LogisticRegression(C=1e5)

# we create an instance of Neighbours Classifier and fit the data.
logreg.fit(X_train_sparse, Y_train)  

LogisticRegression(C=100000.0, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [159]:
Y_test_preds = logreg.predict(X_test_sparse)

In [164]:
Y_test_preds

array([0, 0, 0, ..., 0, 0, 0])

In [165]:
from sklearn.metrics import accuracy_score
accuracy_score(Y_test, Y_test_preds)

0.49725

## Autoencoders

In [ ]:
d.get3DUserTracks(2)

{'_id': {'playlist_pid': 999998}}
{'_id': {'playlist_pid': 999996}}
--Return--
> c:\users\adam khazi\repositories\ucl-dissertation-code\data.py(879)get3DUserTracks()->None
-> pdb.set_trace()


In [ ]:
playlistTrackFeatures